In [2]:
from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
import networkx as nx
import matplotlib.pyplot as plt 
import numpy as np

In [3]:
class Space:
    def __init__(self, name, demand, area, x, y, delivered=0):
        self.name = name
        self.demand = demand
        self.delivered = delivered
        self.x = x
        self.y = y
        self.area = area

    def __str__(self):
        return f"{self.name} - {self.demand}"

In [4]:
class Camion:
    def __init__(self, max_capacity, location, shift_duration):
        self.max_capacity = max_capacity
        self.current_capacity = max_capacity

        self.location = location
        self.routes = []
        self.current_route = []
        
        self.shift_duration = shift_duration
        self.hours = 0

        self.shift_count = 0

In [5]:
class TSP(SearchProblem):
    def init(self, camion):
        self.camion = camion
        self.environment = environment
        self.initial_state = [camion.location, False]
        SearchProblem.__init__(self, self.initial_state)
        self.goal_state = ["P18", True]

    def actions(self, current_state, environment):
        actions = []
        for space in environment:
            if space not in current_state:
                actions.append(space)
        return actions
    
    def result(self, current_state, action):
        current_edge = [current_state[0].name, action]
        
        if action.name == "P18":
            self.camion.current_capacity = self.camion.max_capacity
            self.camion.current_route.append(current_edge)
            self.camion.routes.append(self.camion.current_route)
            self.camion.current_route = []
        else:
            if self.camion.current_capacity >= action.demand - action.delivered:
                # Se puede entregar toda la demanda
                self.camion.current_capacity -= action.demand - action.delivered
                action.delivered = action.demand
            else:
                # No se puede entregar toda la demanda
                action.delivered += self.camion.current_capacity
                self.camion.current_capacity = 0
            
            self.camion.current_route.append(current_edge)

        return [action, all(space.delivered == space.demand for space in self.environment)]
    
    def is_goal(self,state):
        return state == self.goal_state
    
    def cost(self,state1,action,state2):
        distance = (state1[0].x - state2[0].x)**2 + (state1[0].y - state2[0].y)**2
        time = distance / 10 # 10 km/h
        time += 0.5 # Se debe quedar 30 minutos en cada punto.

        return time
        

In [6]:
space_data = {
    # "name" : (x, y, area)
    "P1": (-56, -1298, 5.4),
    "P2": (-602, -1030, 7.52),
    "P3": (-394, -1042, 8),
    "P4": (-195, -1046, 8),
    "P5": (1, -1046, 7.56),
    "P6": (-814, -932, 4.19),
    "P7": (-824, -643, 6.28),
    "P8": (-600, -600, 7.6),
    "P9": (-399, -639, 8),
    "P10": (-195, -632, 8),
    "P11": (0, -600, 7.67),
    "P12": (56, -334, 5.4),
    "P13": (-195, -258, 5.98),
    "P14": (-417, -269, 7.34),
    "P15": (-794, -258, 7.97),
    "P16": (408, -158, 5.64),
    "P17": (189, -43, 6.11),
    "P18": (0, 0, 7.11), # <--------------------------------------- HQ
    "P19": (600, 0, 4.92),
    "P20": (-931, 75, 1.38),
    "P21": (-808, 75, 8),
    "P22": (-604, 79, 7.82),
    "P23": (-403, 136, 5.53),
    "P24": (-206, 133, 5.64),
    "P25": (800, 200, 5.05),
    "P26": (1000, 400, 4.75),
    "P27": (-927, 445, 1.28),
    "P28": (-801, 434, 6.64),
    "P29": (-597, 431, 6.54),
    "P30": (-349, 380, 6.76),
    "P31": (153, -492, 1.47),
}

In [7]:
# No podemos asumir que todos los espacios tienen la misma demanda
# Pero podemos asumir que la demanda de cada espacio es un valor fijo por ahora

spaces = [Space(name=key, demand=10, x=value[0], y=value[1], area=value[2]) for key, value in space_data.items()]
spaces[17].demand = 0

raul = Camion(max_capacity=100, location=spaces[17], shift_duration=8)
# Agent

greedy(TSP(raul, spaces))

TypeError: SearchProblem.__init__() takes from 1 to 2 positional arguments but 3 were given